In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
from torchvision import transforms, datasets
import torchvision

In [2]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
from torchvision import transforms, utils
# ----------------------------
# Sound Dataset
# ----------------------------
class SongLabelSet(Dataset):
  def __init__(self, df):
    self.df = df

  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    values = [
        self.df["acousticness"][idx],
        self.df["danceability"][idx],
        self.df["energy"][idx],
        self.df["instrumentalness"][idx],
        self.df["liveness"][idx],
        self.df["speechiness"][idx],
        self.df["valence"][idx],
    ]
    values = torch.tensor(values, dtype=torch.float32)
    label = self.df["pos_percent"][idx]
    return values, label


In [3]:
songDataset = pd.read_csv("./song_positivity.csv", low_memory=False)

In [4]:
songData = SongLabelSet(songDataset)

In [5]:
len(songData)

4776

In [287]:
songData[2]

(tensor([0.0015, 0.2970, 0.9730, 0.9180, 0.1390, 0.1510, 0.2340]), 0.132678133)

In [8]:
num_items = len(songDataset)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(songData, [num_train, num_val])
num_items
len(train_ds)

3821

In [289]:
batch_size = 32
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [290]:
import torch
import torch.nn as nn

class LabelToPositivityModel(nn.Module):
    def __init__(self):
        super(LabelToPositivityModel, self).__init__()
        self.fc1 = nn.Linear(7, 512)
        self.relu1 = nn.ReLU()

    
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()


        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()


        self.fc4 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
        
        self.fchehe1 = nn.Linear(7,1)



    def forward(self, x):
        x = self.fchehe1(x)
#         x = self.fchehe2(x)
#         x = self.fc1(x)
#         x = self.relu1(x)
        
#         x = self.fc2(x)
#         x = self.relu2(x)

#         x = self.fc3(x)
#         x = self.relu3(x)

#         x = self.fc4(x)
#         x = self.sigmoid(x)

        return x

In [291]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [292]:
model = LabelToPositivityModel()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
criterion = nn.MSELoss()
print(next(model.parameters()).device)

cpu


In [293]:
def trainLabel(model, optimizer, criterion, trainloader, testloader, num_epochs):
    model.train()
    stats = [[],[]]
#     print(device)
    for epoch in range(num_epochs):
        train_loss = 0.0
        val_loss = 0.0
        for data in trainloader:
            
            inputs, labels = data
#             print(inputs)
            labels = labels.type(torch.FloatTensor).unsqueeze(1)
            
            
            optimizer.zero_grad()

            outputs = model(inputs)
#             print(labels)
#             print(outputs)
#             print(outputs.shape)
#             print(labels.shape)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

            

 # Validation loop
        with torch.no_grad():
            for inputs, labels in testloader:
#                 inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.type(torch.FloatTensor).unsqueeze(1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()

        
        # Print epoch statistics
        train_loss /= len(trainloader.dataset)
        stats[0].append(train_loss)

        val_loss /= len(testloader.dataset)
        stats[1].append(val_loss)

        print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}')

    print("Finished training")
    return stats

In [295]:
stats = trainLabel(model, optimizer, criterion, train_dl, val_dl, 40)

Epoch 1: Train Loss=0.0014, Val Loss=0.0014
Epoch 2: Train Loss=0.0014, Val Loss=0.0014
Epoch 3: Train Loss=0.0014, Val Loss=0.0014
Epoch 4: Train Loss=0.0014, Val Loss=0.0014
Epoch 5: Train Loss=0.0014, Val Loss=0.0014
Epoch 6: Train Loss=0.0014, Val Loss=0.0014
Epoch 7: Train Loss=0.0014, Val Loss=0.0014
Epoch 8: Train Loss=0.0014, Val Loss=0.0014
Epoch 9: Train Loss=0.0014, Val Loss=0.0014
Epoch 10: Train Loss=0.0014, Val Loss=0.0014
Epoch 11: Train Loss=0.0014, Val Loss=0.0014
Epoch 12: Train Loss=0.0014, Val Loss=0.0014
Epoch 13: Train Loss=0.0014, Val Loss=0.0014
Epoch 14: Train Loss=0.0014, Val Loss=0.0014
Epoch 15: Train Loss=0.0014, Val Loss=0.0014
Epoch 16: Train Loss=0.0014, Val Loss=0.0014
Epoch 17: Train Loss=0.0014, Val Loss=0.0014
Epoch 18: Train Loss=0.0014, Val Loss=0.0014
Epoch 19: Train Loss=0.0014, Val Loss=0.0014
Epoch 20: Train Loss=0.0014, Val Loss=0.0014
Epoch 21: Train Loss=0.0014, Val Loss=0.0014
Epoch 22: Train Loss=0.0014, Val Loss=0.0014
Epoch 23: Train Los

In [296]:
model.eval()
with torch.no_grad():
    inputt = torch.Tensor(songData[2][0]).unsqueeze(0)
    result = model(inputt)
    print(result)
    print(songData[2][1])

tensor([[0.3756]])
0.132678133


In [297]:
torch.save(model, "./labeltopos.pt")